In [1]:
import matplotlib
import matplotlib.pyplot as plt

import jax
import jax.numpy as np
from jax.random import PRNGKey

import numpyro

import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive

import pandas as pd

import covid
import covid.util as util
import covid.models.SEIRD_variable_detection
import covid.models.SEIRD_bridge
import covid.models.SEIRD_incident

# Run Inference

In [2]:
#data = util.load_state_data()
data = util.load_data()

start = '2020-03-04'
end = '2020-08-02'
model_type = covid.models.SEIRD_incident.SEIRD

#places = ['PR']
places = ['NC-Hertford']
places = ['CO-Mineral']
places = ['NY-New York City']
places = ['NV-Clark']
places = ['MO']

for place in places:
    util.run_place(data,
                   place,
                   start=start,
                   end=end,
                   model_type=model_type,
                   rw_scale=1e-1,
                   resample_high=80,
                   resample_low=0,
                   num_warmup=1000,
                   num_samples=1000)

Running MO (start=2020-03-04, end=2020-08-02)
 * running MCMC


warmup:   8%|▊         | 163/2000 [02:05<23:30,  1.30it/s, 1023 steps of size 3.84e-03. acc. prob=0.76] 


KeyboardInterrupt: 

In [ ]:
#data = util.load_state_data()
data = util.load_county_data()

start = '2020-03-04'
end=None
#places = ['PR']
places = ['NC-Hertford']

save = True

# Inspect and Save Results 
for place in places:
    util.gen_forecasts(data, 
                       place,
                       model_type=model_type,
                       start=start,
                       end=end,
                       save=save)